In [70]:
import json

In [2]:
with open('wino_amrs.jsonl') as f:
    gt_amrs = [json.loads(l) for l in f]

In [107]:
# Change to your output parse file
with open('sample_outputs/wino_blip2-flant5xl_caps_parse.jsonl') as f:
    cand_amrs = [json.loads(l) for l in f]

In [4]:
gt_amrs = gt_amrs[:len(cand_amrs)]

In [95]:
SAMPLE_SIZE=len(cand_amrs[0]['parses_0'])

In [99]:
def full_score(candidate,ground_truth):
    full_score=!python smatch/smatch.py -f {candidate} {ground_truth}  --ms | cut -d ':' -f2
    return list(map(float,full_score))

def full_score_recall(candidate,ground_truth):
    full_score=!python smatch/smatch.py -f {candidate} {ground_truth}  --ms --pr | grep Recall | cut -d ':' -f2
    return list(map(float,full_score))

def rel_score(candidate,ground_truth):
    just_rel=!python smatch/smatch.py -f {candidate} {ground_truth} --ms --justrelation | cut -d ':' -f2
    return list(map(float,just_rel))

def attr_score(candidate,ground_truth):
    just_attr=!python smatch/smatch.py -f {candidate} {ground_truth} --ms --justattribute | cut -d ':' -f2
    return list(map(float,just_attr))

def inst_score(candidate,ground_truth):
    just_inst=!python smatch/smatch.py -f {candidate} {ground_truth} --ms --justinstance | cut -d ':' -f2
    return list(map(float,just_inst))

smatch_score_map = {
    'full':full_score,
    'full_recall':full_score_recall,
    'rel':rel_score,
    'attr':attr_score,
    'inst':inst_score
}

def get_smatch_scores_from_file(ground_truth_path,candidate_path,method='full'):
    if method not in score_map:
        raise ValueError("invalid smatch score method")
    return smatch_score_map[method](candidate_path,ground_truth_path)

def save_tmp_amr(f1,f2,ground_truth_path,candidate_path):
    with open(ground_truth_path,'w') as f, open(candidate_path,'w') as g:
        for p0,p1 in zip(f1,f2):
            print(p0,file=f)
            print(p1,file=g)
            
def get_smatch_scores(f1,f2,method='full'):
    candidate_path="cand_test.amr"
    ground_truth_path="gt_test.amr"
    save_tmp_amr(f1,f2,ground_truth_path,candidate_path)
    return get_smatch_scores_from_file(ground_truth_path,candidate_path,method='full')

In [78]:
import statistics

In [93]:
def get_handles(gt_amrs,cand_amrs):
    fc0=[gt_amr['parse_0'] for gt_amr in gt_amrs for _ in range(SAMPLE_SIZE)]
    fi0=[p for cand_amr in cand_amrs for p in cand_amr['parses_0']]
    fc1=[gt_amr['parse_1'] for gt_amr in gt_amrs for _ in range(SAMPLE_SIZE)]
    fi1=[p for cand_amr in cand_amrs for p in cand_amr['parses_1']]
    return fc0,fc1,fi0,fi1

def get_pair_scores(scoring_func,fc0,fc1,fi0,fi1):
    s00 = list(scoring_func(fc0,fi0))
    s01 = list(scoring_func(fc0,fi1))
    s10 = list(scoring_func(fc0,fi1))
    s11 = list(scoring_func(fc1,fi1))
    return s00,s01,s10,s11

def get_text_score(s00,s01,s10,s11):
    # s00 = score(C0,I0)
    # text score == given image, pick caption
    counts = [c0>x0 and c1>x1 for c0,x0,c1,x1 in zip(s00,s10,s11,s01)]
    return sum(counts)/len(counts),counts

def get_image_score(s00,s01,s10,s11):
    counts = [c0>x0 and c1>x1 for c0,x0,c1,x1 in zip(s00,s01,s11,s10)]
    return sum(counts)/len(counts),counts

def get_group_score(s00,s01,s10,s11):
    counts = [c00>x01 and c00>x10 and c11>x01 and c11>x10 for c00,x01,c11,x10 in zip(s00,s01,s11,s10)]
    return sum(counts)/len(counts),counts

In [103]:
fc0,fc1,fi0,fi1 = get_handles(gt_amrs,cand_amrs)
score_func=lambda f1,f2:get_smatch_scores(f1,f2,method='rel')
s00,s01,s10,s11 = get_pair_scores(score_func,fc0,fc1,fi0,fi1)
text_score,txt_cnt=get_text_score(s00,s01,s10,s11)
im_score,im_cnt=get_image_score(s00,s01,s10,s11)
gp_score,gp_cnt=get_group_score(s00,s01,s10,s11)

In [105]:
print(text_score,im_score,gp_score)

0.085 0.0845 0.0815


In [97]:
s00[0],s10[0],s11[0],s01[0]

(0.54, 0.36, 0.5, 0.36)